# Power Iteration

In this notebook, we look to investigate a linear algebra algorithm called Power Iteration. 

Given a symmetric matrix $A \in \mathbb{R}^{n\times n}$, we can find it's largest eigenvalue by simply taking a random vector $v \in \mathbb{R}^{n}$ and multiplying $A$ with $v$, taking that result, and multiplying $A$ again with it, over and over again. Eventually, the resultant vector will be the eigenvector associated with the largest eigenvalue. 

This eigenvalue can be found via Rayleigh Quotient, defined as follows:
$\lambda = \frac{v^T A v}{v^T v}$


That's right: SIMPLY BY MULTIPLYING A BY A RANDOM VECTOR AND MULTIPLYING THAT OUTPUT BY A AND SO ON, WE CAN FIND THE LARGEST EIGENVALUE/VECTOR! IT'S THAT SIMPLE!

### The Algorithm

Here is the algorithm in words. 

$v_{0}$ = some random vector in $\mathbb{R}^{n}$ with $||v_{0}|| = 1 $

for $k = 1, 2, ... \\
     w = A v_{k-1}  \enspace (Apply \enspace A) \\                  
     v_{k} = w / ||w||  \enspace (Normalize)\\            
     \lambda_{k} = v_{k}^{T} A v_{k} \enspace (Eigenvalue \enspace from \enspace Rayleigh \enspace Quotient)$  

Below is the algorithm in code. 

In [3]:
import numpy as np

def power_iteration(A,v, error_tol = 1e-12, max_iterations = 500):
    """
    Input: 
    A: Matrix whose largest eigenvalue we will find. Numpy array. np.shape(A) = (n,n)
    v: Initial random vector. Numpy array. np.shape(v) = (n,) or (n,1)
    
    Output: 
    lamda: Largest eigenvalue of matrix. float value. 
    v: Eigenvector associated with largest eigenvalue lamda. Numpy array. np.shape(v) = (n,) or (n,1)
    """
    convergence = False
    lamda_previous = 0
    iterations = 0
    error = 1e12 
    
    while (convergence == False):
        w = np.dot(A,v)
        v = w/np.linalg.norm(w) 
        lamda = np.dot(v.T,np.dot(A,v))
        
        #Check convergence 
        error = np.abs(lamda - lamda_previous)/lamda
        iterations += 1 
        
        if (error <= error_tol or iterations >= max_iterations):
            convergence = True
        
        lamda_previous = lamda
        
    return lamda, v

### Example: Compare to Numpy eigh function

In [24]:
#Example 
np.random.seed(100)
#Set dimension of space
n = 10

#Create random matrix in space R^{nxn}
A = np.random.rand(n,n)
#Ensure matrix A is Symmetric 
A = np.dot(A.T,A)
#Create random vector in space R^n
v = np.random.rand(n)

# Find eigenvalues using python built in library. 
# https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html
# The function will return eignevalues in ascending order and eigenvectors correspondingly
eigenvalues, eigenvectors =  np.linalg.eigh(A)
largest_eigenvalue = eigenvalues[-1]
largest_eigenvector = eigenvectors[:,-1]

#Double check with Rayleigh Quotient 
rayleigh_quotient = np.dot(largest_eigenvector.T,np.dot(A,largest_eigenvector))

print('From Numpy built in library.')
print(f'Largest Eigenvalue: {largest_eigenvalue}')
print(f'Largest Eigenvector: {largest_eigenvector}')
# Now try our own code 
largest_eigenvalue_power, largest_eigenvector_power = power_iteration(A,v)
print('From Power Iteration we wrote.')
print(f'Largest Eigenvalue: {largest_eigenvalue_power}')
print(f'Largest Eigenvector: {largest_eigenvector_power}')

From Numpy built in library.
Largest Eigenvalue: 23.197588853558727
Largest Eigenvector: [0.38551468 0.30569119 0.25223855 0.30703594 0.24715369 0.38680035
 0.39724199 0.25227291 0.26081145 0.31602386]
From Power Iteration we wrote.
Largest Eigenvalue: 23.19758885355869
Largest Eigenvector: [0.38551469 0.3056912  0.25223855 0.30703592 0.24715368 0.38680036
 0.39724199 0.2522729  0.26081146 0.31602385]


### Why Does this Work? 

At first glance we can wonder why this works. We simply run $v_{k} = Av_{k-1}$ over and over again and that $v_{k}$ becomes the eigenvector associated with the largest eigenvalue of the matrix $A$. That simple! Let's take a look at the math behind it to see why this happens. 


Our initial vector $v_0$ can be expressed as a linear combination of all eigenvectors in the space $R^n$. 

$v_{0} = a_1 q_1 + q_2 q_2 + ... + a_n q_n$ where $q_i$ are the eigenvectors. If we multiply by $A$, we can use the fact that $Aq_i = \lambda_i q_i$. 

$v_{1} = A (a_1 q_1 + q_2 q_2 + ... + a_n q_n) \\
       = (a_1 Aq_1 + q_2 Aq_2 + ... + a_n Aq_n) \\
       = (a_1 \lambda_1 q_1 + a_2 \lambda_2 q_2 + ... + a_n \lambda_n q_n )
       $


If we do this for $k$ steps, we can get to 

$v_k = c_k (a_1 \lambda_1^k q_1 + a_2 \lambda_2^k q_2 + ... + a_n \lambda_n^k q_n ) $

The constant $c_k$ has come up because we need to account for some lack of normalization of the resulting vectors $v_i$. 

We now have the line:
    
$v_k = c_k (a_1 \lambda_1^k q_1 + a_2 \lambda_2^k q_2 + ... + a_n \lambda_n^k q_n ) $

Let's pull out the leading eigenvalue, which is the largest one. 

$v_k = c_k \lambda_1^k (a_1 q_1 + a_2 (\lambda_2 / \lambda_1)^k q_2 + ... + a_n (\lambda_n / \lambda_1)^k q_n )$

We see that the first term will survive and all other terms will die off eventually so long as the following two condition are met. As $k$ gets large, $(\lambda_i/\lambda_1)^k$ will become very small and therefore die off. We will eventually be left with $v_k = c_k \lambda_1^k a_1 q_1$. Using the Rayleigh Quotient expression on this vector, we can find the eigenvalue corresponding to the normalized vector $v_k = v_k / ||v_k||$. 

We need 

1. $|\lambda_1| > |\lambda_2| > ... |\lambda_n|$ as well as 

2. $q_1^T v_{0} \neq 0$. 

If the former is false, then the later terms will not go to zero as $k$ gets larger. If the latter is false, then the initial random vector will not contain a component corresponding to the first eigenvector. This means we will only be able to find the second largest eigenvalue. 